In [2]:
# importing all the required modules
from tkinter import *
from tkinter import messagebox
import requests
import bs4
import numpy as np



class getRequest:
    extracted_data=[] #creating a class variable to store the data that is extracted in extract_data method
    
    def __init__(self,uid): #gets the result of the given uid 
        
        self.url=f"http://202.53.11.226:8080/lastudentportal/online/report/onlineResultNewInner.jsp?registerno={uid}&iden=1" #url to get the result
        self.res=requests.get(self.url) #using requests modules get method to get the html script
        self.soup=bs4.BeautifulSoup(self.res.text,"lxml") #converting the html script to readable format using bs4 library
        
    def extract_data(self):
        
        self.data=[] #creating an empty list 'data' to store all the data required to display and to calculate gpa
        for i in self.soup.select("tr td"): #all the data present between the <tr> and <td> are selected and given to 'i' 
            self.data.append(i.getText()) #each data item is appended into 'data' list
        
        self.data_1=[] #empty list to store student info  
        self.data_2=[] #empty list to store student result
        self.credits=[] #empty list to store  credits
        self.grade_points=[] #empty list to store grade points
        
        self.gradeValue={'O':10,'A':9,'B':8,'C':7,'D':6,'E':5,'F':0} #dictionery containing grade as keys and grade points as values 
        self.head=np.array([7,8,10,11,12]) #starting index values of the result table
        self.add=np.array([7,7,7,7,7]) #distance between each result row
        
        try:
            for i in [2,4,6]:
                self.data_1.append(self.data[i]) #appending the student info into data_1
            self.sem=self.data[15] # takeing the current semester and storing it to sem    
            for i in range((int((len(self.data)-10)/7))): # iterate the loop until all the table rows are scaned
                
                self.record=[self.data[self.head[0]],self.data[self.head[1]],self.data[self.head[2]],self.data[self.head[3]],self.data[self.head[4]]]
                self.data_2.extend([self.record]) # for every iteration each row is extended into data_2
                
                if self.sem==self.data[self.head[1]]: #check if the current sem is equal to the given semester
                    try:
                        if i!=0: #if this is not the first iteration append the credit values
                            self.credits.append(int(self.data[self.head[3]]))
                    except: #if the credit cell is blank the append a space to the credits
                        self.credits.append(' ')
                        
                for k in self.gradeValue: #taking all the keys in gradeValue and finding match  with grade given in result
                    if self.data[self.head[4]] == k: #if match found append the corresponding grade point
                        self.grade_points.append(self.gradeValue[k])
                    
                self.head=self.head+self.add #incrementing the index position 
                
            self.extracted_data=[self.data_1,self.data_2,self.credits,self.grade_points] #the data to be displayed is stored         
        
        except: #if the result of the entered uid is not present an exception is raised
            self.data_1.append(self.data[0]) #appending the message of result not found and return it  
            
            self.extracted_data=[self.data_1] #the data to be displayed is stored 
        
        
class Window:
    b_c=0 #count how many times the get result button is clicked
        
    def __init__(self): #creating two frames and displaying the entry widget to enter uid 
        
        self.f1=Frame(root,width=800,height=130) # creating frame one
        self.f1.propagate(0)
        self.f1.pack() #packing frame one to the root window
        
        self.f2=Frame(root,width=800,height=350) # creating frame two 
        self.f2.propagate(0)
        self.f2.pack() #packing frame two to the root window
        
        self.uid_l=Label(self.f1,text="ENTER UID") #label widget created
        self.uid_l.place(x=200,y=10) #placed label
        
        self.uid_e=Entry(self.f1) #Entry widget created
        self.uid_e.place(x=280,y=10) #placed entry widget
        self.uid_e.bind('<Return>',self.get_html) #the get_html method is called when enter is clicked
        
        self.sub_b=Button(self.f1,text="Get Result",bg="blue",command=self.get_html) # Button widget created
        self.sub_b.place(x=430,y=7) #placed button
        
        
    def get_html(self,event=None): # when the 'Get Result' button is clicked this get_html funct is clicked
        
        if len(self.uid_e.get()) != 12:#check's whether the given uid is correct or not 
            messagebox.showerror('Invalid UID','Please enter a valid uid')#displayes a messagebox to enter a valid uid
            return # return the control from where it was called
        
        self.uid=self.uid_e.get() # storing the value entered in the entry widget          
        
        if self.b_c!=0: # if the button is clicked for the first time this will return false
            for widget in root.winfo_children(): #getting all the widgets of the root window i.e. f1 and f2 
                for w in widget.winfo_children(): #all the widgets present on the frames are stored to w 
                    if w not in [self.uid_l, self.uid_e,self.sub_b]: #except the widget present in the list are destroyed
                        w.destroy() #destroying the widget present in w
            
        self.b_c+=1 # every time when the button is clicked the count is incremented   
        
        self.request=getRequest(self.uid) #creating an object to getRequest class and passing the uid as parameter
        
        self.request.extract_data() #calling the extract_data method using the object of getRequest method 
        
        self.studentInfo(self.request.extracted_data[0])#calling the studentInfo method of Window class to display student information
                   
    def studentInfo(self,data):# method to display student info
           
        if len(data)==3: #if the student info is present in data it is diplayed
            
            self.name_l=Label(self.f1,text=f"Name:{data[0]}") #display student name
            self.name_l.place(x=10,y=50) #place name label
            
            self.uids_l=Label(self.f1,text=f"UID : {data[1]}") #display uid label
            self.uids_l.place(x=10,y=70) #place uid label
            
            self.course_l=Label(self.f1,text=f"Course : {data[2]}") #display course label
            self.course_l.place(x=10,y=90) #place course label
            
            self.resultDisplay(self.request.extracted_data[1])#calling the resultDisplay method of Window class to display result
            
            self.calculateGPA(self.request.extracted_data[2],self.request.extracted_data[3])#calling the calculateGPA method of Window class to display the calculated gpa
            
        else:#else the result not here message is displayed
            self.noResult_l=Label(self.f1,text=f"{data[0]}") #display result not available message
            self.noResult_l.place(x=190,y=40) #place noResult label
        
        
    def resultDisplay(self,result): # method to display student result in the form of table
        
        self.n_r=len(result) #getting number of rows in result table
        self.n_c=len(result[0]) #getting number of columns in each row
        
        for i in range(self.n_r): #iterates through all the rows
            
            for j in range(self.n_c): #iterates through each column
                
                if j==2 and i==0: # to display the subject description in center
                    
                    self.tab_l=Label(self.f2,text=result[i][j],width=69,height=2,borderwidth=2,relief='ridge',justify="center")
                    self.tab_l.grid(row=i,column=j)                    
                    
                elif j==2 : #to display all the subject names aligned to left  
                    
                    self.tab_l=Label(self.f2,text=result[i][j],width=69,height=2,borderwidth=2,relief='ridge',anchor="w")
                    self.tab_l.grid(row=i,column=j)
                    
                else: #to display remaining values 
                                
                    self.tab_l=Label(self.f2,text=result[i][j],width=10,height=2,borderwidth=2,relief='ridge',justify="center")
                    self.tab_l.grid(row=i,column=j)
                
                  
                    
    def calculateGPA(self,credits,grade_points): # method to calculate gpa of the given uid
        
        credits=np.array(credits) # converting credits list to numpy array
        grade_points=np.array(grade_points) #converting grade points list to numpy array
        self.gpa_l=Label(self.f2) #creating label to display the calculate gpa
        self.gpa_l.grid(column=2) #placing the label on the grid
    
        try: #try to calculate gpa
            self.gpa=(sum(credits*grade_points)/sum(credits)) # calculate gpa using credits and grade points 
            self.gpa_l.config(text=f"YOUR GPA OF SEM - {self.request.sem} IS {round(self.gpa,2)}") #dispaly the calculated gpa
        except: #if could not calculate gpa 
            self.gpa_l.config(text=f"COULD NOT CALCULATE YOUR GPA OF SEM - {self.request.sem}") #dispaly that could not calculate gpa
           
        
if __name__ == '__main__': # execution starts from here
    root=Tk() # creating parent window
    root.title("Loyola Results") # giving title to parent window
    root.geometry("800x600") # specificing the default size of the root window

    start=Window()# creating an object of Window class

    root.mainloop() # looping the root window infinitely            
   
